In [ ]:
! pip install mediapipe
! pip install sk-video
import numpy as np 
import pandas as pd 
import skvideo.io
import pandas as pd
import numpy as np
import cv2 
import mediapipe as mp
import skvideo.io
import matplotlib.pyplot as plt
import os
import time

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat May  1 12:32:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! wget https://zenodo.org/record/4010759/files/Days_and_Time_3of3.zip

In [ ]:
! unzip Days_and_Time_3of3.zip

In [ ]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

def pose_estimation(image,results):
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        return image

In [ ]:
for dirpath,dirname,file in os.walk("./Days_and_Time"):
   for i in dirname:
    os.makedirs(os.path.join("./Output/",i))

In [ ]:
def video_array_maker(pather,height=224,width=224):
  videodata = skvideo.io.vread(pather)  
  outpath=os.path.join("./Output",pather.split("/")[2],os.path.split(pather)[1])
  out = cv2.VideoWriter(outpath,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))
  #start=time.time()

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#    arr=[]
    actualframe=len(videodata)
    print("Actual frame {}".format(actualframe))
    if actualframe >=45:
          for i in range (actualframe):
            x=round (actualframe/(45)  * i)
            if x >=actualframe:
                    break
            else:
                frame =videodata[x]
                frame =cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
                results = holistic.process(frame)  

                output = pose_estimation(frame,results)
                output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
#                arr.append(output)
                out.write(output)               
    else:
          for i in range(actualframe):
              frame=videodata[i]
              frame=cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
              results = holistic.process(frame)  
              output = pose_estimation(frame,results)
              output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
 #             arr.append(output)
              out.write(output)
          for i in range(45-actualframe):
              
              newframe=np.zeros(shape=(height,width,3))
              
#              arr.append(newframe)
              out.write(np.uint8(newframe))
    out.release()
    print("File Created : {}".format(outpath))
    #end=time.time()
    #print(end-start)
    #arr=np.array(arr)/255.

In [ ]:
for dirpathed,dirnamed,file in os.walk("./Days_and_Time"):
   z=0
   for class_ in dirnamed:
        for dirpath,dirname,files in os.walk(os.path.join(dirpathed,class_)):
              for i in files:
                    pather=os.path.join(dirpath,i)
                    z+=1
                    print("................ {} th Video...........".format(z))
                    video_array_maker(pather)
        print(".............{} over..........".format(class_))

In [ ]:
! pip install keras-video-generators
import os
import glob
import keras
from keras_video import VideoFrameGenerator
# use sub directories names as classes
import tensorflow

In [ ]:
classes = [i.split(os.path.sep)[2] for i in glob.glob('./Output/*')]
classes

['84. Evening',
 '86. Time',
 '81. Second',
 '83. Afternoon',
 '82. Morning',
 '85. Night']

In [ ]:
data_aug = tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,horizontal_flip=True,width_shift_range=3.0,
                                                                   height_shift_range=3.0,brightness_range=(0.2,1),)

In [ ]:

# ImageDataGenerator
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10, # rotation
        width_shift_range=0.3, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.5,1.5]) # brightness

In [ ]:

# some global params
SIZE = (224,224)
CHANNELS = 3
NBFRAME = 45
BS = 2
# pattern to get videos and classes
glob_pattern='./Output/{classname}/*'
# for data augmentation

# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.2, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=datagen,
    use_frame_cache=False)

class 81. Second, validation count: 3, train count: 12
class 82. Morning, validation count: 2, train count: 12
class 83. Afternoon, validation count: 2, train count: 12
class 84. Evening, validation count: 2, train count: 12
class 85. Night, validation count: 3, train count: 12
class 86. Time, validation count: 3, train count: 12
Total data: 6 classes for 72 files for train


In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout,Input

In [ ]:
Model=None

In [ ]:
mobilenet = tf.keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=(224,224,3),
        weights='imagenet')
    # Keep 9 layers to train
trainable = 14
for layer in mobilenet.layers[:-trainable]:
    layer.trainable = False
for layer in mobilenet.layers[-trainable:]:
    layer.trainable = True

In [ ]:
model=Sequential()
model.add(TimeDistributed(mobilenet ,input_shape=(45,224,224,3)))
model.add(TimeDistributed( Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False)) 
model.add(Dense(64,activation="relu"))
model.add(Dense(6,activation="softmax")) 
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=tf.keras.metrics.Accuracy())

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_12 (TimeDis (None, 45, 7, 7, 1024)    3228864   
_________________________________________________________________
time_distributed_13 (TimeDis (None, 45, 50176)         0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               25756160  
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 390       
Total params: 28,993,670
Trainable params: 27,358,662
Non-trainable params: 1,635,008
_________________________________________________________________


In [ ]:
valid = train.get_validation_generator()

Total data: 6 classes for 15 files for validation


In [ ]:
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=20)

Epoch 1/20
36/36 [==============================] - 47s 1s/step - loss: 1012.9003 - accuracy: 0.5712 - val_loss: 3068.8098 - val_accuracy: 0.6667
Epoch 2/20
36/36 [==============================] - 43s 1s/step - loss: 5374.8769 - accuracy: 0.6792 - val_loss: 28174.1113 - val_accuracy: 0.7381
Epoch 3/20
36/36 [==============================] - 43s 1s/step - loss: 27580.9482 - accuracy: 0.7023 - val_loss: 23613.4121 - val_accuracy: 0.7143
Epoch 4/20
36/36 [==============================] - 43s 1s/step - loss: 7001.0860 - accuracy: 0.7704 - val_loss: 6383.9175 - val_accuracy: 0.7381
Epoch 5/20
36/36 [==============================] - 43s 1s/step - loss: 5598.2159 - accuracy: 0.6960 - val_loss: 5641.5952 - val_accuracy: 0.7143
Epoch 6/20
36/36 [==============================] - 43s 1s/step - loss: 4613.1318 - accuracy: 0.6941 - val_loss: 6458.4888 - val_accuracy: 0.6786
Epoch 7/20
36/36 [==============================] - 43s 1s/step - loss: 2388.8708 - accuracy: 0.7111 - val_loss: 1406.344

In [ ]:
model.save("6classes")

INFO:tensorflow:Assets written to: 6classes/assets


In [ ]:
model.save("my_model1",save_format="tf")

INFO:tensorflow:Assets written to: my_model1/assets


In [ ]:
checkmodel = keras.models.load_model('/content/6classes')

In [ ]:
import os
 
file_size = os.path.getsize('/content/6classes/variables/variables.data-00000-of-00001')
print("File Size is :", file_size, "bytes")

File Size is : 334904662 bytes


In [ ]:
334904662/1000000

334.904662

# Deploying Section :

In [ ]:
def video_array_maker(pather,height=224,width=224):
  videodata = skvideo.io.vread(pather)  
  
  #outpath=os.path.join("./Output",pather.split("/")[2],os.path.split(pather)[1])
  outpath=os.path.join("./test",pather.split("/")[2],os.path.split(pather)[1])
  out = cv2.VideoWriter(outpath,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))
  #start=time.time()

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#    arr=[]
    actualframe=len(videodata)
    print("Actual frame {}".format(actualframe))
    if actualframe >=45:
          for i in range (actualframe):
            x=round (actualframe/(45)  * i)
            if x >=actualframe:
                    break
            else:
                frame =videodata[x]
                frame =cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
                results = holistic.process(frame)  

                output = pose_estimation(frame,results)
                output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
#                arr.append(output)
                out.write(output)               
    else:
          for i in range(actualframe):
              frame=videodata[i]
              frame=cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
              results = holistic.process(frame)  
              output = pose_estimation(frame,results)
              output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
 #             arr.append(output)
              out.write(output)
          for i in range(45-actualframe):
              
              newframe=np.zeros(shape=(height,width,3))
              
#              arr.append(newframe)
              out.write(np.uint8(newframe))
    out.release()
    print("File Created : {}".format(outpath))
    #end=time.time()
    #print(end-start)
    #arr=np.array(arr)/255.

In [ ]:
aftpath="/content/afternoon.mp4"
video_array_maker(aftpath)
morpath="/content/morning.mp4"
video_array_maker(morpath)

Actual frame 77
File Created : ./test/afternoon.mp4/afternoon.mp4
Actual frame 81
File Created : ./test/morning.mp4/morning.mp4


In [ ]:
#os.makedirs("./test/afternoon.mp4")
os.makedirs("./test/time.mp4")

In [ ]:
timepath="/content/time.mp4"
video_array_maker(timepath)

Actual frame 100
File Created : ./test/time.mp4/time.mp4


In [ ]:
aft =skvideo.io.vread("./test/afternoon.mp4/afternoon.mp4")
aft.shape

(45, 224, 224, 3)

In [ ]:
mor =skvideo.io.vread("/content/test/morning.mp4/morning.mp4")
mor.shape

(45, 224, 224, 3)

In [ ]:
time=skvideo.io.vread("/content/test/time.mp4/time.mp4")
time.shape

(45, 224, 224, 3)

In [ ]:
a=[]
a.append(aft)
a.append(mor)
a.append(time)

In [ ]:
a=np.array(a)
a.shape

(3, 45, 224, 224, 3)

In [ ]:
x=model.predict(a)

In [ ]:
x

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
train.classes

['81. Second',
 '82. Morning',
 '83. Afternoon',
 '84. Evening',
 '85. Night',
 '86. Time']

In [ ]:
classes

['81. Second',
 '82. Morning',
 '83. Afternoon',
 '84. Evening',
 '85. Night',
 '86. Time']